In [1]:
import re
import nltk
import string
import warnings
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
import time

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# train = pd.read_csv('Compiled_train_data_1.csv')
train = pd.read_csv('Compiled_train_data_1.csv', encoding='utf-8')
print(train.head(10))


                                               tweet  label
0  Natumai kwamba, nitakapojiwekea akiba, nitawez...      1
1  2020 की शुरुआत में लॉक डाउन से पहले दौरा किया ...      1
2  2020 kee shuruaat mein lok daun se pahale daur...      1
3                       It was good, Bahut maza aaya      1
4  Fortunate enough for visiting this temple for ...      1
5  Have visited before lock down early 2020 and t...      1
6  Peaceful place. Gives calmness to one who is i...      1
7  Love this place . We have been there for many ...      1
8  It was a fantastic atmosphere. So.much colour ...      1
9  Good place to visit with your family and frien...      1


In [3]:
from englisttohindi.englisttohindi import EngtoHindi
from langdetect import DetectorFactory
DetectorFactory.seed = 0

from langdetect import detect
from langdetect import detect_langs
from google_trans_new import google_translator  

#simple function to detect and translate text 
def detect_and_translate(text,target_lang):
    
    result_lang = detect_langs(text)
    # print(result_lang[0])
    if result_lang[0].lang == target_lang:
        return text 
    
    else:        
        if result_lang[0].prob<=0.999:
            text=EngtoHindi(message=text)
            text=text.convert
            
        translator = google_translator()
        translate_text = translator.translate(text,lang_tgt=target_lang)
        return translate_text 

In [5]:
from englisttohindi.englisttohindi import EngtoHindi
mess=[[] for j in range(5)]
mess[0]="2020 kee shuruaat mein lok daun se pahale daura kiya hai aur baingalor ke daure ke dauraan yah avashy jaana chaahie. aashcharyajanak mandir aur adbhut vaastukala"
mess[1]="2020 की शुरुआत में लॉक डाउन से पहले दौरा किया है और बैंगलोर के दौरे के दौरान यह अवश्य जाना चाहिए। आश्चर्यजनक मंदिर और अद्भुत वास्तुकला"
mess[2]="It was good. Bahut maza aaya, mein firse aana chahunga"
mess[3]="Will surely go again and seek blessings from lord Krishna"
mess[4]="Natumai kwamba, nitakapojiwekea akiba, nitaweza kusafiri kwenda Mexico"

# mess=[a.replace(".",",") for a in mess]
# mess=[a.replace("[^a-zA-Z#]", " ") for a in mess]
mess=[' '.join([word.strip(string.punctuation) for word in message.split(" ")]) for message in mess]

print(detect_and_translate(mess[0],'en'))
print(detect_and_translate(mess[1],'en'))
print(detect_and_translate(mess[2],'en'))
print(detect_and_translate(mess[3],'en'))
print(detect_and_translate(mess[4],'en'))

At the beginning of 2020, there is a tour before Public Doon and the visit of the Bangalore or the Awanous Temple and Amazing Architecture 
At the beginning of 2020, lock down has been visited and it must be done during the visit of Bangalore. Amazing temple and wonderful architecture 
I would like to come again 
Will surely go again and seek blessings from lord Krishna
I hope that I have saved I can travel to Mexico 


In [ ]:
train['tweet']=[' '.join([word.strip(string.punctuation) for word in message.split(" ")]) for message in train['tweet']]
for i in range(0,len(train)):
    train['tweet'][i] = detect_and_translate(train['tweet'][i],'en')
    if i%400==0:
        print(i)
        time.sleep(1)
train.to_csv('prosecced_data2.csv')

In [6]:
train.to_csv('prosecced_data2.csv')

In [14]:
train.head(10)

In [3]:
train[train['label'] == 0].head(10)

,tweet,label
6,We visited yesterday morning to be told that a...,0
7,good place to visit iskon temole is a must wat...,0
54,"Expect to get jostled, ushered and shoved but ...",0
74,Over-hyped temple. May be a special place for ...,0
81,located on a hill the temple is built on a sma...,0
84,This is on higher level situated\nVery long wa...,0
85,It ia very nice to visit! Wear socks!!! You wi...,0
91,Strange place. I was looking for a spiritual p...,0
95,A magnificent piece of architecture that is ne...,0
99,It's a huge temple. You have to leave your sho...,0


In [4]:
train[train['label'] == 1].head(10)

,tweet,label
0,Fortunate enough for visiting this temple for ...,1
1,Have visited before lock down early 2020 and t...,1
2,Peaceful place. Gives calmness to one who is i...,1
3,Love this place . We have been there for many ...,1
4,It was a fantastic atmosphere. So.much colour ...,1
5,Good place to visit with your family and frien...,1
8,I really approve Iskcon for one thing: I heard...,1
9,Very peaceful place. Loved the bhajan and arti...,1
10,This is one place in bangalore which I visit o...,1
11,"Worth visiting, I just hoped they had better c...",1


In [5]:
train.shape

(8193, 2)

In [6]:
train['label'].value_counts()

1    7159
0    1034
Name: label, dtype: int64

In [7]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

In [ ]:
Combine= train
Combine['tweets'] = np.vectorize(remove_pattern)(Combine['tweet'], '@[\w]*')
Combine['tweets'] = Combine['tweets'].str.replace("[^a-zA-Z#]", " ")
Combine.head(10)

In [ ]:
Combine['tweets'] = Combine['tweets'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
Combine.head(10)

In [ ]:
Tokenize_tweet = Combine['tweets'].apply(lambda x: x.split())
Tokenize_tweet.head(10)

In [ ]:
from nltk.stem.porter import *

stemmer = PorterStemmer()
Tokenize_tweet = Tokenize_tweet.apply(lambda x: [stemmer.stem(i) for i in x])

In [ ]:
#nltk’s MosesDetokenizer function

for i in range(len(Tokenize_tweet)):
    Tokenize_tweet[i] = ' '.join(Tokenize_tweet[i])
    
Combine['tweets'] = Tokenize_tweet

In [ ]:
Combine.head(10)

In [ ]:

all_words = ' '.join([text for text in Combine['tweets']])

from wordcloud import WordCloud
wordcloud = WordCloud(width = 800, height = 600, random_state = 21, max_font_size = 120).generate(all_words)

plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import gensim

In [ ]:
BOWvectorize = CountVectorizer(max_df=0.90, min_df=2, max_features = 1000, stop_words='english')
BOW = BOWvectorize.fit_transform(Combine['tweets'])
BOW.shape

In [ ]:
TfidfVect = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
Tfidf = TfidfVect.fit_transform(Combine['tweets'])
Tfidf.shape

In [ ]:
Tokenize_tweet = Combine['tweets'].apply(lambda x: x.split())

Model_W2V = gensim.models.Word2Vec(Tokenize_tweet, vector_size=200, # No. of features
                                   window=5, 
                                   min_count=2, 
                                   sg=1, # 1 for skip-gram model
                                   hs=0,
                                   negative=10, # for negative sampling
                                   workers=2,  # No. of cores
                                   seed=34 )

Model_W2V.train(Tokenize_tweet, total_examples=len(Combine['tweets']), epochs=20)

In [ ]:
def word_vec(tokens, size):
  vec = np.zeros(size).reshape((1, size))
  count = 0
  for word in tokens:
    try:
      vec += Model_W2V.wv[word].reshape((1, size))
      count += 1
    except KeyError:
      
      continue
    
  if count != 0:
    vec = vec/count
   
  return vec

In [ ]:
Arrays = np.zeros((len(Tokenize_tweet), 200))

for i in range(len(Tokenize_tweet)):
  Arrays[i,:] = word_vec(Tokenize_tweet[i], 200)
  
Arr_df = pd.DataFrame(Arrays)
Arr_df.shape

## Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [ ]:
#Extract train and test bow features
Train_Bow = BOW
#Test_Bow = BOW[7589:,:]

#Split data into train and validation set
XTrain_Bow, XValid_Bow, YTrain, YValid = train_test_split(Train_Bow, train['label'],
                                                          random_state = 42,
                                                          test_size=0.2)

In [ ]:

LR = LogisticRegression(solver='lbfgs', max_iter=500, multi_class='multinomial')
LR.fit(XTrain_Bow, YTrain)

Prediction = LR.predict_proba(XValid_Bow)

# if prediction is greater than or equal to 0.3 than 1 else 0
Pred = Prediction[:,1] >= 0.3
Pred = Pred.astype(np.int)

f1_score(YValid, Pred, average='micro')

In [ ]:
Train_Tfidf = Tfidf
#Test_Tfidf = Tfidf[7589:,:]

XTrain_tfidf = Train_Tfidf[YTrain.index]
XValid_tfidf = Train_Tfidf[YValid.index]

In [ ]:
LR.fit(XTrain_tfidf, YTrain)

Prediction = LR.predict_proba(XValid_tfidf)

Pred = Prediction[:,1] >= 0.3
Pred = Pred.astype(np.int)

f1_score(YValid, Pred, average='micro')

In [ ]:
Train_W2V = Arr_df
#Test_W2V = Arr_df.iloc[7589:,:]

XTrain_w2v = Train_W2V.iloc[YTrain.index,:]
XValid_w2v = Train_W2V.iloc[YValid.index,:]

In [ ]:
LR.fit(XTrain_w2v, YTrain)

Prediction = LR.predict_proba(XValid_w2v)

Pred = Prediction[:,1] >= 0.3
Pred = Pred.astype(np.int)

f1_score(YValid, Pred, average='micro')

## SVM

In [ ]:
from sklearn import svm

In [ ]:
svc = svm.SVC(kernel='linear', C=1, probability=True)
svc.fit(XTrain_Bow, YTrain)

Prediction = svc.predict_proba(XValid_Bow)

Pred = Prediction[:,1] >= 0.3
Pred = Pred.astype(np.int)
f1_score(YValid, Pred, average='micro')

In [ ]:
svc.fit(XTrain_tfidf, YTrain)

Prediction = svc.predict_proba(XValid_tfidf)
Pred = Prediction[:,1] >= 0.3
Pred = Pred.astype(np.int)
f1_score(YValid, Pred, average='micro')

In [ ]:
svc.fit(XTrain_w2v, YTrain)

Prediction = svc.predict_proba(XValid_w2v)

Pred = Prediction[:,1] >= 0.3
Pred = Pred.astype(np.int)
f1_score(YValid, Pred, average='micro')

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=11)

rf.fit(XTrain_Bow, YTrain)

Prediction = rf.predict(XValid_Bow)
f1_score(YValid, Prediction, average='micro')

In [ ]:
rf.fit(XTrain_tfidf, YTrain)

Prediction = rf.predict(XValid_tfidf)
f1_score(YValid, Prediction, average='micro')

In [ ]:
rf.fit(XTrain_w2v, YTrain)

Prediction = rf.predict(XValid_w2v)
f1_score(YValid, Prediction, average='micro')

## Extreme Gradient Boosting (XGBoost)

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgbModel = XGBClassifier(Max_depth=6, n_estimators=1000)

xgbModel.fit(XTrain_Bow, YTrain)

Prediction = xgbModel.predict(XValid_Bow)
f1_score(YValid, Prediction, average='micro')

In [ ]:
xgbModel.fit(XTrain_tfidf, YTrain)

Prediction = xgbModel.predict(XValid_tfidf)
f1_score(YValid, Prediction, average='micro')

In [ ]:
xgbModel = XGBClassifier(Max_depth=6, n_estimators=1000, nthread=3)
xgbModel.fit(XTrain_w2v, YTrain)

Prediction = xgbModel.predict(XValid_w2v)
f1_score(YValid, Prediction, average='micro')